# Peer-graded Assignment: 
## Segmenting and Clustering Neighborhoods in Toronto

# 1. Scraping information 

based on information on wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M a pandas dataframe with columns
    * PostalCode
    * Borough
    * Neighborhood
   

The wikipedia list is a list of postal codes in Canada where the first letter is M. 
Postal codes beginning with M are located within the city of Toronto in the province of Ontario.

### documentation
https://beautiful-soup-4.readthedocs.io/en/latest/#

 see https://www.youtube.com/watch?v=ng2o98k983k for use of beautifullsoup
    

In [1]:
# with open('simple.html') as html_file:
#    soup = BeautifulSoup(html_file, 'lxml')
# print(soup) ## the entire code, not indented   
# print(soup.prettify()) ## indented

# match = soup.title  ## finds the title tag (including tags)
# match = soup.title.text
# print(match)

# match = soup.div
# match = soup.find('div')   ## results the same (13:00)
# match = soup.find('div', class_='footer')  ## finds a certain (here underscore because class is python specific word

# article = soup.find('div', class_='article')
# headline = article.h2.a.text
# summary = article.p.text

# for article in soup.find_all('div', class_='article'):
#    headline = article.h2.a.text
#    summary = article.p.text


Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20181105055415-0001


### libraries install


In [ ]:
## in anaconda prompt:  
## conda install -c conda-forge beautifulsoup4 --yes
## conda install -c conda-forge lxml --yes   # 4:33
## conda install -c conda-forge html5lib --yes   # not done
##
## conda install -c conda-forge geocoder --yes   # not done, too much changes in the environment
#!conda install -c esri arcgis --verbose --yes
#!conda install -c conda-forge folium=0.5.0 --verbose --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/ibm/conda/miniconda3

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge

The following packages will be DOWNGRADED:

    folium: 0.6.0-py_0 conda-forge --> 0.5.0-py_0 conda-forge

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... ===> UNLINKING PACKAGE: conda-forge::folium-0.6.0-py_0 <===
  prefix=/opt/ibm/conda/miniconda3


===> LINKING PACKAGE: conda-forge::folium-0.5.0-py_0 <===
  prefix=/opt/ibm/conda/miniconda3
  source=/home/spark/shared/user-libs/python3/folium-0.5.0-py_0


done


### libraries import


In [37]:
from bs4 import BeautifulSoup 
import requests
import pandas as pd
import numpy as np

from arcgis.gis import GIS
#from arcgis.gis import *
from arcgis.geocoding import geocode, reverse_geocode
#from arcgis.geometry import Point

import folium

### data scraping

In [41]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text
soup_raw = BeautifulSoup(source, 'html5lib')
print(soup_raw.prettify())
print("done")

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":862527922,"wgRevisionId":862527922,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wg

In [42]:
soup_table = soup_raw.tbody
# print(soup_table)
## parser has added columnnames into tbody while wikipediasite had a thead-tag
print("done")

done


In [43]:
## building the dataframe from the soup_table result
scraped_result = []
#result = pd.DataFrame(columns = ["PostalCode"])
scraped_result = pd.DataFrame(columns = ["PostalCode","Borough","Neighborhood"])
for soup_tr in soup_table.find_all('tr'):
    i = 1 
    ## for empty records in tablerows where there is no table data (like the table header line) a dummy result is given
    PostalCode = "A1A"
    Borough = "Not assigned"
    Neighborhood = "Not assigned"
    for  soup_td in soup_tr.find_all('td'):
        if i == 3:
            Neighborhood = str(soup_td.text)
            Neighborhood = Neighborhood.replace("\n","")
        if i == 2:
            Borough = str(soup_td.text)
            i = 3
        if i == 1:
            PostalCode = str(soup_td.text)
            i = 2
    newline = pd.DataFrame([[PostalCode,Borough,Neighborhood]], columns = ["PostalCode","Borough","Neighborhood"] )
    scraped_result = scraped_result.append(newline, ignore_index=True)

scraped_result.head()

,PostalCode,Borough,Neighborhood
0,A1A,Not assigned,Not assigned
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


### cleaning data

***Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.***

In [44]:
## removing lines with "Not assigned" in Borough, this includes eventual dummy records
cleaning1 = scraped_result.copy()
cleaning1 = cleaning1.loc[cleaning1['Borough'] != "Not assigned"]
#cleaning1
print('done !')

done !


***If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
So, the value of the Borough AND the Neighborhood columns will be 'Queen's Park'***

In [45]:
## where "not assigned" in Neighborhood copy the name of the borough
## assumption is made that this never applies for PostalCodes which cover multiple Neighborhoods, 
## hence the order of cleaning can be different than in the 'rules' of the assignment 
cleaning2 = cleaning1.copy()
## splitting the dataset in two parts, not to be adjusted, and to adjusted
cleaning2a = cleaning2.loc[cleaning2['Neighborhood'] != "Not assigned"]
cleaning2b = cleaning2.loc[cleaning2['Neighborhood'] == "Not assigned"]
cleaning2b['Neighborhood'] = cleaning2b['Borough'] 
## previous line induced the following error: 
##     "A value is trying to be set on a copy of a slice from a DataFrame.
##     Try using .loc[row_indexer,col_indexer] = value instead
##     See the caveats in the documentation: 
##     http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy"
## In this case it is a false positive: 
## see https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas#20627316

## and combining the two parts
cleaning2 = cleaning2a.append(cleaning2b)
print('done !')
# cleaning2


done !


/opt/ibm/conda/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


***More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.***

In [46]:
## the dataframe cleaning3 is buildup by adding lines from previous step (cleaning2)
cleaning3 = pd.DataFrame(columns = ["PostalCode", "Borough", "Neighborhood"])
for postalcode in set(cleaning2['PostalCode']):
    tempstring = ''
    no_of_neighborhoods = cleaning2[cleaning2['PostalCode'] == postalcode].shape[0]
    ## making a small dataframe with just the current postalcode
    tempdata = cleaning2[cleaning2['PostalCode'] == postalcode]
    tempdata = tempdata.reset_index()
    for i in range(0,no_of_neighborhoods):
        ## the assumption is made that each PostalCode, even when it compasses more than one Neighborhood
        ## it only compasses only one Borough. so it doesn't matter from which line the borough information comes
        borough = tempdata.loc[i,'Borough']
        strippednb = tempdata.loc[i,'Neighborhood'].strip()
        tempstring = tempstring + strippednb 
        if i < no_of_neighborhoods-1:
            tempstring = tempstring + ", "
    newline = pd.DataFrame([[postalcode,borough,tempstring]], columns = ["PostalCode","Borough","Neighborhood"] )
    cleaning3 = cleaning3.append(newline)

cleaning3 = cleaning3.reset_index(drop = True)
print('done !')
#cleaning3 


done !


In the center there are some "neighborhoods" which turn out to be (large) officebuildings, airports or stations. 
The following PostalCodes are removed in further analysis   
    * CFB Toronto (M3K)
    * Union station (M5J)
    * Design Exchange, Toronto Dominion Centre, Downtown Toronto (M5K)
    * Commerce Court, Victoria Hotel (M5L)
    * Stn A PO Boxes 25 The Esplanade (M5W)
    * Canada Post Gateway Processing Centre (M7R), by the way This lies outside the city border (western side)
    * Business reply mail Processing Centre969 Eastern, East Toronto (M7Y)
    
*"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara"* is kept because South Niagara is an actual Neighborhood.

*"first canadian place, underground city"* is kept because undergroud city a.k.a. "PATH" is the worlds largest underground 'sidewalk' with 18 miles of walkways and 1200 shops (https://en.wikipedia.org/wiki/Underground_city#Canada)

Upper Rouge, M1X has no results from foursquare with setting 850

It will also be removed from the set

In [47]:
toronto_data_adj =  cleaning3.copy()
#geodataframe_adj = geodataframe_adj[geodataframe_adj['PostalCode'] isin({'M3K','M5J','M5K','M5L','M5W','M7R','M7Y'})][:]
toronto_data_adj = toronto_data_adj[toronto_data_adj['PostalCode'] != 'M3K']
toronto_data_adj = toronto_data_adj[toronto_data_adj['PostalCode'] != 'M5J']
toronto_data_adj = toronto_data_adj[toronto_data_adj['PostalCode'] != 'M5K']
toronto_data_adj = toronto_data_adj[toronto_data_adj['PostalCode'] != 'M5L']
toronto_data_adj = toronto_data_adj[toronto_data_adj['PostalCode'] != 'M5W']
toronto_data_adj = toronto_data_adj[toronto_data_adj['PostalCode'] != 'M7R']
toronto_data_adj = toronto_data_adj[toronto_data_adj['PostalCode'] != 'M7Y']
## and Upper Rouge:
toronto_data_adj = toronto_data_adj[toronto_data_adj['PostalCode'] != 'M1X']
toronto_data_adj = toronto_data_adj.reset_index(drop = True)
print('{} records are left \n done !'.format(toronto_data_adj.shape[0]))


95 records are left 
 done !


In [48]:
toronto_data = toronto_data_adj.copy()


## results

In [49]:
toronto_data

,PostalCode,Borough,Neighborhood
0,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."
1,M4J,East York,East Toronto
2,M4G,East York,Leaside
3,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
4,M4E,East Toronto,The Beaches
5,M6N,York,"The Junction North, Runnymede"
6,M9M,North York,"Emery, Humberlea"
7,M1T,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter"
8,M4P,Central Toronto,Davisville North
9,M2J,North York,"Fairview, Henry Farm, Oriole"


In [50]:
toronto_data.shape

(95, 3)

## 2. Getting geo-information

since installing of geocoder required a substantial change to my local environment including DOWNGRADES  of packages I opted for using ArcGIS environment. geocoding and reverse geocoding can be done anonymous.

### libraries install 
for a good overview of the notebook installing libraries is done in the preamble of this notebook.

    * from arcgis.gis import GIS
    * #from arcgis.gis import#
    * from arcgis.geocoding import geocode, reverse_geocode
    * #from arcgis.geometry import Point

## retrieving the information
getting information about neighborhoods is done in a series of calls to the api. 
I will try if the first 3 (alphanumerics) of the PostalCode will yield an answer

Log into ArcGIS Online as an anonymous user.
Simple geocoding and reverse geocoding do not require credits, hence you need not log in using your credentials.

In [51]:
dev_gis = GIS()

In [52]:
#geocode(address="Bathorst Manor")[0]
#geocode(address ="M3H", search_extent=None, location=None, distance=None, out_sr=None, category=None, out_fields='*', max_locations=10, magic_key=None, for_storage=False, geocoder=None, as_featureset=False)

In [53]:
geodataframe = toronto_data.copy()
geodataframe['Latitude'] = 0
geodataframe['Longitude'] = 0

In [54]:
for i in range(geodataframe.shape[0]):
    postalcode = geodataframe.loc[i,'PostalCode']
    queryresult = geocode(address=postalcode,search_extent=None, out_fields='location', max_locations=1, as_featureset=False)
    longitude = queryresult[0]['location']['x'] 
    latitude = queryresult[0]['location']['y']
    geodataframe.loc[i, 'Longitude'] = longitude
    geodataframe.loc[i, 'Latitude'] = latitude
print('done !')

done !


In [55]:
resultB=geodataframe.copy()

## Results

In [56]:
resultB

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.632765,-79.489601
1,M4J,East York,East Toronto,43.688765,-79.334175
2,M4G,East York,Leaside,43.709765,-79.363791
3,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.686810,-79.557284
4,M4E,East Toronto,The Beaches,43.676531,-79.295410
5,M6N,York,"The Junction North, Runnymede",43.675795,-79.481960
6,M9M,North York,"Emery, Humberlea",43.733665,-79.537477
7,M1T,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter",43.784725,-79.299244
8,M4P,Central Toronto,Davisville North,43.712755,-79.388514
9,M2J,North York,"Fairview, Henry Farm, Oriole",43.780810,-79.347782


## Part 3 analysis

getting a feel for Toronto by plotting a map and the Neighborhoods.

A circle of radius 500 is plotted around each neighborhood to see if a call to foursquare with this radius is covering the whole neighborhood

In [57]:
### libraries install 
#for a good overview of the notebook installing libraries is done in the preamble of this notebook.

## Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

## map rendering library
import folium 

# library to handle requests
import requests 

## libraries to handle and convert JSON files
import json 
from pandas.io.json import json_normalize 

## import k-means from clustering stage
from sklearn.cluster import KMeans


In [58]:
## replotting in housecolors of van Zanden actuarial services
# create map of Toronto using latitude and longitude values
latitude = 43.72 
longitude =-79.45 
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11,  width='60%', height='60%' )
# add markers to map
for lat, lng, borough, neighborhood in zip(resultB['Latitude'], resultB['Longitude'], resultB['Borough'], resultB['Neighborhood']):
    label = '{}: {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#005578',
        fill=True,
        fill_color='#2fac66',
        fill_opacity=0.7#,
        #parse_html=False
        ).add_to(map_toronto)  
    
map_toronto


### getting foursquare information

In [59]:
# The code was removed by Watson Studio for sharing.

A run with LIMIT = 100 and radius = 500 resulted in only 1908 (an average of less than 20 venues per neighborhood)

The risk of increasing the radius is double counting of venues. 
For this reason the function getNearbyVenues() is adapted so that the venue postalcode is retrieved. This can be matched with the postalcode of the neighborhood of which the lat and lng are submitted in the foursquare call. 

Postalcode is not available for all venues, hence the rewrite of a part of the code in getNearbyVenues(). In these cases it is assumed the location is a part of the Neighborhood and gets the first 3 characters of the Neighborhood PostalCode and the fictious (?) suffix ' 0Z0'

In [60]:
def getNearbyVenues(postals, names, latitudes, longitudes, radius, limit):
    ## for testing this function only a limited amount of calls is needed, to reduce risk of reaching daily limits on foursquare
    blndebug = False
    intdebugcount = 1
    intdebugmax = 3
    if blndebug == True:
        limit = 15
    venues_list=[]
    for nbpostal, name, lat, lng in zip(postals, names, latitudes, longitudes):
        #print(name)
        if intdebugcount/10 == int(intdebugcount/10):
            print(intdebugcount)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, 
            lat, lng, radius, 
            limit)
            
        # make the GET request, with error handling split retreiving data and filtering to [][][]['items']
        if blndebug == False:
            results_raw = requests.get(url).json()
        elif intdebugcount <= intdebugmax:
            results_raw = requests.get(url).json()
        else:
            results_raw = results_raw # raw results from last item
        intdebugcount =  intdebugcount + 1    
        
        results = results_raw.copy()
        try:
            results = results["response"]['groups'][0]['items']
        except KeyError:
            print("--- ")
            print("function getNearbyVenues (internal):")
            print("A KeyError occured when trying results = results['response']['groups'][0]['items'] from the following 'results':")
            print(results)
            #print("---")
        
        # temporary code to find empty results
        #print(results)    
        #print()
        #input('{}: var results from {}, {} is printed above'.format(intdebugcount,name, nbpostal))
        
        for v in results:
            vname = v['venue']['name']
            vid = v['venue']['id']
            vlat =v['venue']['location']['lat']
            vlng = v['venue']['location']['lng']
            try:
                vpc6 =v['venue']['location']['postalCode']
                pcapprox = False
            except KeyError:
                vpc6 = nbpostal + ' 0Z0'
                pcapprox = True
            vpc3 = vpc6[:3]
            vcat = v['venue']['categories'][0]['name']
            venues_list.append([(name, nbpostal, lat, lng,
                                vname, vid,
                                vlat, vlng, vpc6, vpc3, pcapprox,
                                vcat)])
    if blndebug == True:
        print(venues_list)            
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                   'Neighborhood PostalCode',           
                   'Neighborhood Latitude', 
                   'Neighborhood Longitude', 
                   'Venue',
                   'Venue id', ## new in 2nd run
                   'Venue Latitude', 
                   'Venue Longitude',
                   'Venue PostalCode6',
                   'Venue PostalCode3', 
                   'Venue PcApproximation',          
                   'Venue Category']
    
    return(nearby_venues)

In [61]:
radius =850
LIMIT = 150

In [62]:
## getting the data (with a backup of last results, to be able to continue working when connection fails or qouta is exceeded)
try:
    toronto_venues_raw_backup = toronto_venues_raw.copy()
    print("A backup could be made")
    blnBackupAvailable = True
except:
    print("no backup could be made")
    blnBackupAvailable = False
try:
    toronto_venues_raw = getNearbyVenues(postals = resultB['PostalCode'],names= resultB['Neighborhood'],latitudes=resultB['Latitude'],longitudes= resultB['Longitude'], radius = radius, limit = LIMIT)
    print('done !')
except TypeError:
    print("---")
    print("a TypeError occured in function getNearbyVenues: ")
    if blnBackupAvailable == True:
        print("no update of toronto_venues_raw, Luckyly there is a backup")
        #print("---")  
    else:
        print("TROUBLE: no update of toronto_venues_raw, and NO backup")
        #print("---")  


no backup could be made
10
20
30
40
50
60
70
80
90
done !


In [63]:
## when a backup has to be restored: uncomment next line
# toronto_venues_raw = toronto_venues_raw_backup.copy()

In [64]:
print(toronto_venues_raw.shape)
#toronto_venues_raw.head()

(3794, 12)


In [65]:
## remove results where Neighborhood Postalcode is not equal to Venue PostalCode3
toronto_venues = toronto_venues_raw.copy()
toronto_venues = toronto_venues.loc[toronto_venues['Neighborhood PostalCode'] == toronto_venues['Venue PostalCode3'], ]
toronto_venues.shape

(3070, 12)

In [66]:
toronto_venues.head()

,Neighborhood,Neighborhood PostalCode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue id,Venue Latitude,Venue Longitude,Venue PostalCode6,Venue PostalCode3,Venue PcApproximation,Venue Category
0,"Humber Bay, King's Mill Park, Kingsway Park So...",M8Y,43.632765,-79.489601,Starbucks,551486ff498ee91ac6d40f43,43.627894,-79.489275,M8Y 3H8,M8Y,False,Coffee Shop
1,"Humber Bay, King's Mill Park, Kingsway Park So...",M8Y,43.632765,-79.489601,Tim Hortons,4e5f0010814d9e0233e0b715,43.628255,-79.489590,M8Y 3H8,M8Y,False,Coffee Shop
2,"Humber Bay, King's Mill Park, Kingsway Park So...",M8Y,43.632765,-79.489601,Tom's Dairy Freeze,4adf75b3f964a520c87a21e3,43.627897,-79.496638,M8Y 0Z0,M8Y,True,Ice Cream Shop
3,"Humber Bay, King's Mill Park, Kingsway Park So...",M8Y,43.632765,-79.489601,Roccos Plum Tomato,4b8da8def964a520000733e3,43.630335,-79.485012,M8Y 1J2,M8Y,False,Italian Restaurant
4,"Humber Bay, King's Mill Park, Kingsway Park So...",M8Y,43.632765,-79.489601,LCBO,4b12cc4df964a5205e8e23e3,43.630094,-79.481985,M8Y 1H6,M8Y,False,Liquor Store


In [67]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 316 uniques categories.


In [68]:
overlap = toronto_venues_raw.shape[0]- toronto_venues.shape[0]
multipresent = toronto_venues.shape[0]-len(toronto_venues['Venue id'].unique())
total = toronto_venues.shape[0]
perc = multipresent / total *100
print('radius :' + str(radius))
print('known overlap : ' + str(overlap))
print('total valid :' + str(total))
print("a total of {} venues are multiple times present (due to missing postalcode) out of {}, this is {} percent of the total".format(
    multipresent, total, perc))


radius :850
known overlap : 724
total valid :3070
a total of 76 venues are multiple times present (due to missing postalcode) out of 3070, this is 2.4755700325732897 percent of the total


The results of several runs of different radius resulted in the following results.  When increasing the radius over 850 the double counting (of venues of which no individual postalcode is known) starts to exceed 2.5%, which I consider an acceptable percentage. No investigation is made how these double countings are spread over different Neigborhoods, which is left as a point of interest for future (week 4 and 5) examinations of the capstone project 

radius  - unique -  known overlap - total valid - multiple - percent
      * 500- 243 -345 - 1563 - 26 - 1.6
      * 750 - 295 - 673 -2674 - 55 - 2
      * 850 - 316 - 724 - 3070 - 76 - 2.47 
      * 900 - 317 - 795 - 3197 - 97 - 3.03
      * 1000 - 319 - 954 -3568 - 146 - 4
      
The analysis is proceeding with results from the 850 m limit call to foursquare.      

In [69]:
toronto_venues = toronto_venues.reset_index(drop = True)

## analyzing each neigborhood


In [70]:
## removing redundant columns before analysis (ba)
toronto_venues_ba = toronto_venues.copy()
toronto_venues_ba = toronto_venues_ba.loc[:,{"Neighborhood",
                                            "Neighborhood Latitude",
                                            "Neighborhood Longitude",
                                            "Venue",
                                            "Venue Latitude",
                                            "Venue Longitude",
                                            "Venue Category"}]
toronto_venues_ba.head()
#toronto_venues_ba2 = toronto_venues_ba[{'Neighborhood','Neighborhood Latitude','Neighborhood Longitude','Venue','Venue Latitude','Venue Longitude','Venue Category'}]

,Venue Category,Venue,Venue Longitude,Neighborhood,Venue Latitude,Neighborhood Longitude,Neighborhood Latitude
0,Coffee Shop,Starbucks,-79.489275,"Humber Bay, King's Mill Park, Kingsway Park So...",43.627894,-79.489601,43.632765
1,Coffee Shop,Tim Hortons,-79.489590,"Humber Bay, King's Mill Park, Kingsway Park So...",43.628255,-79.489601,43.632765
2,Ice Cream Shop,Tom's Dairy Freeze,-79.496638,"Humber Bay, King's Mill Park, Kingsway Park So...",43.627897,-79.489601,43.632765
3,Italian Restaurant,Roccos Plum Tomato,-79.485012,"Humber Bay, King's Mill Park, Kingsway Park So...",43.630335,-79.489601,43.632765
4,Liquor Store,LCBO,-79.481985,"Humber Bay, King's Mill Park, Kingsway Park So...",43.630094,-79.489601,43.632765


In [71]:
toronto_onehot = []
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues_ba[['Venue Category']], prefix="", prefix_sep="")
print(toronto_onehot.columns)
 
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues_ba['Neighborhood'] 
#print(toronto_onehot['Neighborhood'])
print(toronto_onehot.shape)
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

#toronto_onehot.head()

Index(['Adult Boutique', 'Afghan Restaurant', 'African Restaurant',
       'American Restaurant', 'Animal Shelter', 'Antique Shop', 'Arcade',
       'Arepa Restaurant', 'Argentinian Restaurant', 'Art Gallery',
       ...
       'Video Game Store', 'Video Store', 'Vietnamese Restaurant',
       'Warehouse Store', 'Wine Bar', 'Wine Shop', 'Wings Joint',
       'Women's Store', 'Yoga Studio', 'Zoo Exhibit'],
      dtype='object', length=316)
(3070, 316)


In [72]:
toronto_onehot.shape

(3070, 316)

In [73]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo Exhibit,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.00,0.000000,0.000000,0.0,0.041667,0.0,0.000000,0.0,0.000000,...,0.020833,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,Agincourt,0.00,0.000000,0.000000,0.0,0.022727,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.022727,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
4,"Alderwood, Long Branch",0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
5,"Bathurst Manor, Downsview North, Wilson Heights",0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
6,Bayview Village,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
7,"Bedford Park, Lawrence Manor East",0.00,0.000000,0.000000,0.0,0.033333,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
8,Berczy Park,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.017241,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
9,"Birch Cliff, Cliffside West",0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


In [74]:
toronto_grouped.shape

(95, 316)

In [75]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
              venue  freq
0              Café  0.08
1        Steakhouse  0.06
2       Coffee Shop  0.06
3             Hotel  0.06
4  Asian Restaurant  0.04


----Agincourt----
                venue  freq
0  Chinese Restaurant  0.16
1          Restaurant  0.05
2       Shopping Mall  0.05
3         Supermarket  0.05
4    Malay Restaurant  0.02


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                  venue  freq
0                  Park  0.25
1              Pharmacy  0.25
2  Caribbean Restaurant  0.12
3           Pizza Place  0.12
4                   Gym  0.12


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0        Grocery Store  0.13
1          Pizza Place  0.13
2          Coffee Shop  0.13
3  Fried Chicken Joint  0.07
4             Pharmacy  0.07


----Alderwood, Long Branch----
           venue  freq
0    Pizza Place  0.11
1  

In [76]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [77]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Café,Steakhouse,Coffee Shop,Hotel,Bakery,American Restaurant,Bar,Asian Restaurant,Pizza Place,Burrito Place
1,Agincourt,Chinese Restaurant,Restaurant,Shopping Mall,Supermarket,Pharmacy,Coffee Shop,Breakfast Spot,Bubble Tea Shop,Lounge,Filipino Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Pharmacy,Caribbean Restaurant,Pizza Place,Shop & Service,Gym,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Pizza Place,Coffee Shop,Grocery Store,Discount Store,Beer Store,Japanese Restaurant,Park,Sandwich Place,Fried Chicken Joint,Hardware Store
4,"Alderwood, Long Branch",Pizza Place,Skating Rink,Grocery Store,Garden Center,Gas Station,Bank,Beer Store,Liquor Store,Sandwich Place,Dance Studio
5,"Bathurst Manor, Downsview North, Wilson Heights",Pizza Place,Coffee Shop,Bank,Deli / Bodega,Men's Store,Middle Eastern Restaurant,Pharmacy,Sandwich Place,Fried Chicken Joint,Frozen Yogurt Shop
6,Bayview Village,Bank,Chinese Restaurant,Café,Japanese Restaurant,Trail,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space
7,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Fast Food Restaurant,Sushi Restaurant,Butcher,Café,Pet Store,Sandwich Place,Liquor Store,Restaurant
8,Berczy Park,Café,Coffee Shop,Bakery,Cocktail Bar,Restaurant,Seafood Restaurant,Farmers Market,Cheese Shop,Beer Bar,Park
9,"Birch Cliff, Cliffside West",Park,Hotel,College Stadium,Skating Rink,Café,General Entertainment,Gym,Gym Pool,Yoga Studio,Event Space


In [78]:
neighborhoods_venues_sorted.shape

(95, 11)

## clustering

In [79]:
toronto_merged = resultB.copy()
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.632765,-79.489601
1,M4J,East York,East Toronto,43.688765,-79.334175
2,M4G,East York,Leaside,43.709765,-79.363791
3,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.686810,-79.557284
4,M4E,East Toronto,The Beaches,43.676531,-79.295410


In [80]:
# set number of clusters
kclusters = 10

In [81]:
# run k-means clustering
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood' , axis = 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 7, 4, 0, 0, 0, 0, 0, 7, 4, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 4,
       4, 0, 0, 0, 6, 0, 0, 0, 8, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0,
       7, 7, 4, 0, 1, 9, 7, 0, 0, 0, 0, 0, 0, 0, 9, 0, 7, 0, 0, 0, 0, 0, 7,
       0, 7, 2, 5, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0,
       0, 0, 0], dtype=int32)

In [82]:
# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
#toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels
0,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.632765,-79.489601,0
1,M4J,East York,East Toronto,43.688765,-79.334175,0
2,M4G,East York,Leaside,43.709765,-79.363791,7
3,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.686810,-79.557284,4
4,M4E,East Toronto,The Beaches,43.676531,-79.295410,0


In [83]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Even when increasing the number of clusters to 10 most Neighborhoods fall in the same cluster. This means that Toronto Neighborhoods are very similar in possibilities. This might have something to do with the climate. If travelling is, or can be, difficult then all needed venues are more spreadout and possible of smaller scale. 